# **Практическое задание 9**

## Задание 1

In [1]:
!apt-get update -y
!apt-get install -y openmpi-bin libopenmpi-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,881 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,648 kB]
Get:14 http://security.u

In [2]:
%%writefile program.cpp
#include <mpi.h>           // MPI
#include <iostream>        // cout
#include <vector>          // vector
#include <random>          // random
#include <cmath>           // sqrt
#include <algorithm>       // max

using namespace std;

int main(int argc, char** argv) {

    // Инициализация MPI
    MPI_Init(&argc, &argv);

    int rank, size;                          // rank = номер процесса, size = количество процессов
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Размер массива (можешь менять)
    const long long N = 1'000'000;

    // Засекаем время в начале (по инструкции)
    double start_time = MPI_Wtime();

    // base = сколько элементов получит каждый гарантированно
    long long base = N / size;

    // rem = сколько элементов "остатка" нужно распределить по 1 элементу
    long long rem = N % size;

    // На rank 0 будет полный массив данных
    vector<double> data;

    if (rank == 0) {
        data.resize(N);

        // Генератор случайных чисел (фиксируем seed для воспроизводимости)
        mt19937 rng(42);
        uniform_real_distribution<double> dist(0.0, 1.0);

        // Заполняем массив случайными числами
        for (long long i = 0; i < N; i++) {
            data[i] = dist(rng);
        }
    }

    // Каждый процесс принимает базовую часть размером base
    vector<double> local(base);

    // 1) Раздаём base элементов каждому процессу через MPI_Scatter
    // sendbuf значим только на rank 0, остальные передают nullptr
    MPI_Scatter(
        (rank == 0 ? data.data() : nullptr), // откуда отправляем (только rank 0)
        (int)base,                           // сколько элементов каждому
        MPI_DOUBLE,                          // тип данных
        local.data(),                        // куда принимаем
        (int)base,                           // сколько принимаем
        MPI_DOUBLE,                          // тип данных
        0,                                   // root
        MPI_COMM_WORLD                       // коммуникатор
    );

    // 2) Учитываем остаток: первые rem процессов получают +1 элемент
    // Для них расширяем local на 1 и докидываем один элемент
    if (rank < rem) {
        local.resize(base + 1);              // увеличиваем local, чтобы поместить extra

        if (rank == 0) {
            // rank 0 просто берёт extra элемент из своего массива
            // Индекс extra начинается после base*size
            local[base] = data[base * size + rank];
        } else {
            // Остальные получают extra через MPI_Recv от rank 0
            MPI_Recv(
                &local[base],                // куда положить 1 элемент
                1,                           // количество
                MPI_DOUBLE,                  // тип
                0,                           // от кого (root)
                123,                         // tag
                MPI_COMM_WORLD,              // коммуникатор
                MPI_STATUS_IGNORE            // статус не нужен
            );
        }
    } else {
        // Эти процессы получили ровно base элементов, local уже правильного размера
        // local.size() == base
    }

    // root отправляет extra элементы тем процессам, которым они нужны (rank 1..rem-1)
    if (rank == 0) {
        for (int r = 1; r < rem; r++) {
            double extra = data[base * size + r]; // extra элемент для процесса r
            MPI_Send(
                &extra,                         // что отправляем
                1,                              // 1 элемент
                MPI_DOUBLE,                     // тип
                r,                              // кому
                123,                            // tag
                MPI_COMM_WORLD                  // коммуникатор
            );
        }
    }

    // 3) Каждый процесс считает локальные суммы
    double local_sum = 0.0;
    double local_sumsq = 0.0;

    for (double x : local) {
        local_sum += x;           // сумма элементов
        local_sumsq += x * x;     // сумма квадратов
    }

    // 4) Собираем суммы на rank 0 через MPI_Reduce
    double global_sum = 0.0;
    double global_sumsq = 0.0;

    MPI_Reduce(&local_sum, &global_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_sumsq, &global_sumsq, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    // Засекаем время в конце (по инструкции)
    double end_time = MPI_Wtime();

    // 5) Rank 0 считает mean и std и печатает
    if (rank == 0) {
        double mean = global_sum / (double)N;

        // variance = (1/N)*sum(x^2) - ((1/N)*sum(x))^2
        double variance = (global_sumsq / (double)N) - (mean * mean);

        // Из-за погрешностей variance может стать чуть отрицательной (например -1e-16)
        variance = max(0.0, variance);

        double stddev = sqrt(variance);

        cout << "N = " << N << ", processes = " << size << endl;
        cout << "Mean: " << mean << endl;
        cout << "Std dev: " << stddev << endl;
        cout << "Execution time: " << (end_time - start_time) << " seconds." << endl;
    }

    // Завершение MPI
    MPI_Finalize();
    return 0;
}

Writing program.cpp


In [3]:
!mpic++ program.cpp -O2 -o program

In [4]:
!mpirun --allow-run-as-root  --oversubscribe -np 2 ./program
!mpirun --allow-run-as-root  --oversubscribe -np 4 ./program
!mpirun --allow-run-as-root  --oversubscribe -np 8 ./program

N = 1000000, processes = 2
Mean: 0.500055
Std dev: 0.288622
Execution time: 0.0489197 seconds.
N = 1000000, processes = 4
Mean: 0.500055
Std dev: 0.288622
Execution time: 0.118118 seconds.
N = 1000000, processes = 8
Mean: 0.500055
Std dev: 0.288622
Execution time: 0.125014 seconds.


## **Вывод**

В ходе выполнения задания была разработана MPI-программа для распределённого вычисления среднего значения и стандартного отклонения массива случайных чисел размером N=1000000. Генерация массива выполнялась на процессе с рангом 0, после чего данные были распределены между процессами с помощью функции MPI_Scatter с учётом остатка при делении массива. Каждый процесс вычислял локальную сумму элементов и сумму квадратов, которые затем собирались на процессе rank = 0 с использованием MPI_Reduce. На основе глобальных сумм были рассчитаны среднее значение и стандартное отклонение.

Полученные результаты:

* При np=2:
Execution time ≈ 0.049 с

* При np=4:
Execution time ≈ 0.118 с

* При np=8:
Execution time ≈ 0.125 с

Среднее значение массива во всех экспериментах составило около 0.500055, а стандартное отклонение около 0.288622, что соответствует ожидаемым значениям для равномерного распределения случайных чисел на интервале [0,1]. Это подтверждает корректность реализации вычислений и согласованность результатов при различном количестве процессов.

Анализ времени выполнения показывает, что увеличение числа процессов не привело к ускорению программы. Наоборот, при росте количества процессов общее время выполнения увеличилось. Это объясняется тем, что для данной задачи вычислительная нагрузка на каждый процесс относительно невелика, а накладные расходы на коммуникации (операции MPI_Scatter, MPI_Send/Recv, MPI_Reduce) начинают доминировать над временем самих вычислений. При большом числе процессов затраты на обмен данными и синхронизацию возрастают, что снижает эффективность масштабирования.

Таким образом, распределённые вычисления наиболее оправданы для более тяжёлых задач или значительно больших объёмов данных, где выигрыш от параллельной обработки превышает накладные расходы передачи данных между процессами.

## Задание 2

In [5]:
%%writefile program2.cpp
#include <mpi.h>                  // подключаю MPI, чтобы работать с mpirun и процессами
#include <iostream>               // для cout и вывода на экран
#include <vector>                 // для удобных динамических массивов vector
#include <random>                 // чтобы генерировать случайные числа
#include <cmath>                  // для fabs (модуль) и деления/проверок
#include <algorithm>              // для min/max

using namespace std;              // чтобы не писать std:: каждый раз

// CUDA тут не нужна, поэтому только MPI версия метода Гаусса

int main(int argc, char** argv) {                 // main принимает argc/argv, потому что так принято в MPI

    MPI_Init(&argc, &argv);                       // запускаю MPI окружение

    int rank;                                     // номер текущего процесса
    int size;                                     // сколько всего процессов
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);         // получаю rank
    MPI_Comm_size(MPI_COMM_WORLD, &size);         // получаю size

    const int N = 8;                              // по твоей просьбе делаю фиксированный размер N=8

    double start_time;                            // переменная для времени старта
    double end_time;                              // переменная для времени конца

    vector<double> A;                             // матрица A будет храниться полностью только на rank 0
    vector<double> b;                             // вектор b тоже полностью только на rank 0

    int base = N / size;                          // сколько строк гарантированно достанется каждому процессу
    int rem  = N % size;                          // остаток строк, которые распределяются по 1 строке

    int local_rows = base + (rank < rem ? 1 : 0); // если rank меньше rem, то он получает +1 строку

    int start_row;                                // глобальный индекс первой строки, принадлежащей этому процессу
    if (rank < rem) {                             // если процесс среди первых rem
        start_row = rank * (base + 1);            // его блоки длиннее на 1 строку
    } else {                                      // иначе
        start_row = rem * (base + 1) + (rank - rem) * base; // вычисляю сдвиг после "длинных" блоков
    }

    if (rank == 0) {                              // только root создаёт матрицу и вектор
        A.resize(N * N);                          // выделяю память под матрицу N x N
        b.resize(N);                              // выделяю память под вектор b

        mt19937 rng(42);                          // фиксированный seed, чтобы результаты были одинаковые
        uniform_real_distribution<double> dist(0.0, 1.0); // равномерные числа [0,1]

        for (int i = 0; i < N; i++) {             // иду по строкам
            double rowsum = 0.0;                  // сумма модулей строки, чтобы сделать диагональное доминирование

            for (int j = 0; j < N; j++) {         // иду по столбцам
                double val = dist(rng);           // генерирую случайный элемент
                A[i * N + j] = val;               // записываю в A
                rowsum += fabs(val);              // добавляю модуль в сумму строки
            }

            A[i * N + i] = rowsum + N;            // усиливаю диагональный элемент, чтобы pivot не был маленьким
            b[i] = dist(rng);                     // генерирую элемент правой части
        }
    }

    vector<double> localA(local_rows * N, 0.0);   // локальная часть матрицы (local_rows строк)
    vector<double> localb(local_rows, 0.0);       // локальная часть вектора b

    MPI_Barrier(MPI_COMM_WORLD);                  // синхронизация, чтобы честно начать измерение времени
    start_time = MPI_Wtime();                     // старт времени

    // дальше я хочу распределить строки между процессами через MPI_Scatter
    // но MPI_Scatter умеет раздавать только одинаковое количество строк
    // поэтому я раздаю base строк каждому, а остаток rem добрасываю вручную send/recv

    if (base > 0) {                               // если base=0 (когда процессов больше чем строк), Scatter будет бессмысленный
        vector<double> scatterA(base * N);        // временный буфер для base строк матрицы
        vector<double> scatterb(base);            // временный буфер для base элементов b

        MPI_Scatter(                              // раздаю base строк матрицы
            (rank == 0 ? A.data() : nullptr),     // root отправляет A, остальные nullptr
            base * N,                             // сколько элементов double отправляется каждому
            MPI_DOUBLE,                           // тип данных
            scatterA.data(),                      // куда принимаю
            base * N,                             // сколько принимаю
            MPI_DOUBLE,                           // тип данных
            0,                                    // root
            MPI_COMM_WORLD                        // коммуникатор
        );

        MPI_Scatter(                              // раздаю base элементов вектора b
            (rank == 0 ? b.data() : nullptr),     // root отправляет b
            base,                                 // сколько элементов каждому
            MPI_DOUBLE,                           // тип
            scatterb.data(),                      // куда принимаю
            base,                                 // сколько принимаю
            MPI_DOUBLE,                           // тип
            0,                                    // root
            MPI_COMM_WORLD                        // коммуникатор
        );

        for (int i = 0; i < base; i++) {          // копирую base строк в localA/localb
            for (int j = 0; j < N; j++) {         // копирую строку целиком
                localA[i * N + j] = scatterA[i * N + j]; // перенос элемента
            }
            localb[i] = scatterb[i];              // перенос соответствующего b
        }
    }

    // теперь распределяю остаток rem строк
    // каждая из этих строк идёт процессам rank=0..rem-1 как дополнительная строка

    if (rank < rem) {                             // только процессы, которые получают extra строку
        int local_extra = base;                   // extra строка сохраняется после base строк, то есть индекс base

        if (rank == 0) {                          // root свою extra строку просто копирует сам
            int global_row = base * size + rank;  // глобальный индекс extra строки
            for (int j = 0; j < N; j++) {         // копирую всю строку
                localA[local_extra * N + j] = A[global_row * N + j];
            }
            localb[local_extra] = b[global_row];  // копирую соответствующий элемент b
        } else {                                  // остальные получают extra строку через Recv
            MPI_Recv(                             // принимаю строку матрицы
                &localA[local_extra * N],         // куда писать
                N,                                // сколько элементов
                MPI_DOUBLE,                       // тип
                0,                                // от root
                200,                              // tag
                MPI_COMM_WORLD,                   // коммуникатор
                MPI_STATUS_IGNORE                 // статус не нужен
            );

            MPI_Recv(                             // принимаю b для этой строки
                &localb[local_extra],             // куда писать
                1,                                // 1 элемент
                MPI_DOUBLE,                       // тип
                0,                                // от root
                201,                              // tag
                MPI_COMM_WORLD,                   // коммуникатор
                MPI_STATUS_IGNORE                 // статус
            );
        }
    }

    if (rank == 0) {                              // root отправляет extra строки процессам 1..rem-1
        for (int r = 1; r < rem; r++) {           // только тем, кому нужен extra
            int global_row = base * size + r;     // индекс extra строки для процесса r

            MPI_Send(                             // отправляю строку
                &A[global_row * N],               // адрес начала строки
                N,                                // N элементов
                MPI_DOUBLE,                       // тип
                r,                                // кому
                200,                              // tag
                MPI_COMM_WORLD                    // коммуникатор
            );

            MPI_Send(                             // отправляю b элемент
                &b[global_row],                   // адрес элемента
                1,                                // 1 элемент
                MPI_DOUBLE,                       // тип
                r,                                // кому
                201,                              // tag
                MPI_COMM_WORLD                    // коммуникатор
            );
        }
    }

    vector<double> pivotRow(N, 0.0);              // буфер для pivot строки, которую будем рассылать
    double pivotB = 0.0;                          // pivot элемент вектора b

    // прямой ход метода Гаусса
    for (int k = 0; k < N; k++) {                 // k это текущий столбец и строка pivot

        int owner = -1;                           // owner это процесс, у которого хранится строка k

        for (int r = 0; r < size; r++) {          // ищу owner перебором процессов
            int r_rows = base + (r < rem ? 1 : 0);// сколько строк у процесса r
            int r_start;                          // стартовая строка процесса r
            if (r < rem) r_start = r * (base + 1);// старт для процессов с extra строкой
            else r_start = rem * (base + 1) + (r - rem) * base; // старт для остальных

            if (k >= r_start && k < r_start + r_rows) { // если k попадает в диапазон
                owner = r;                         // значит owner найден
                break;                             // выхожу из цикла
            }
        }

        if (rank == owner) {                       // только владелец строки k готовит pivotRow
            int local_k = k - start_row;           // локальный индекс этой строки в localA
            for (int j = 0; j < N; j++) {          // копирую pivot строку
                pivotRow[j] = localA[local_k * N + j];
            }
            pivotB = localb[local_k];              // копирую pivot элемент b
        }

        MPI_Bcast(pivotRow.data(), N, MPI_DOUBLE, owner, MPI_COMM_WORLD); // рассылаю pivotRow всем
        MPI_Bcast(&pivotB, 1, MPI_DOUBLE, owner, MPI_COMM_WORLD);        // рассылаю pivotB всем

        double pivot = pivotRow[k];                // pivot элемент (диагональ) для деления

        for (int li = 0; li < local_rows; li++) {  // иду по локальным строкам процесса
            int global_i = start_row + li;         // считаю глобальный индекс строки

            if (global_i <= k) continue;           // беру только строки ниже pivot строки

            double factor = localA[li * N + k] / pivot; // коэффициент для вычитания

            localA[li * N + k] = 0.0;              // зануляю элемент под диагональю

            for (int j = k + 1; j < N; j++) {      // обновляю остальные элементы строки
                localA[li * N + j] -= factor * pivotRow[j]; // вычитаю factor * pivotRow
            }

            localb[li] -= factor * pivotB;         // обновляю правую часть
        }
    }

    // теперь собираю матрицу и вектор обратно на rank 0, чтобы сделать обратный ход
    vector<int> recvcountsA;                       // сколько элементов A приходит от каждого процесса
    vector<int> displsA;                           // смещения в итоговом массиве A
    vector<int> recvcountsB;                       // сколько элементов b приходит от каждого процесса
    vector<int> displsB;                           // смещения в итоговом векторе b

    if (rank == 0) {                               // эти массивы нужны только root
        recvcountsA.resize(size);
        displsA.resize(size);
        recvcountsB.resize(size);
        displsB.resize(size);

        int offA = 0;                              // текущее смещение по A
        int offB = 0;                              // текущее смещение по b

        for (int r = 0; r < size; r++) {           // для каждого процесса считаю размеры
            int r_rows = base + (r < rem ? 1 : 0); // строки процесса r
            recvcountsA[r] = r_rows * N;           // сколько doubles матрицы от процесса r
            displsA[r] = offA;                     // куда класть
            offA += recvcountsA[r];                // сдвигаю оффсет

            recvcountsB[r] = r_rows;               // сколько doubles b от процесса r
            displsB[r] = offB;                     // куда класть
            offB += recvcountsB[r];                // сдвигаю оффсет
        }

        A.assign(N * N, 0.0);                      // выделяю заново место под A (уже верхнетреугольная)
        b.assign(N, 0.0);                          // выделяю заново место под b (обновленный)
    }

    MPI_Gatherv(                                   // собираю localA на root
        localA.data(),                             // что отправляю
        local_rows * N,                            // сколько отправляю
        MPI_DOUBLE,                                // тип
        (rank == 0 ? A.data() : nullptr),          // куда собирать на root
        (rank == 0 ? recvcountsA.data() : nullptr),// массив количеств
        (rank == 0 ? displsA.data() : nullptr),    // массив смещений
        MPI_DOUBLE,                                // тип
        0,                                         // root
        MPI_COMM_WORLD                             // коммуникатор
    );

    MPI_Gatherv(                                   // собираю localb на root
        localb.data(),                             // что отправляю
        local_rows,                                // сколько отправляю
        MPI_DOUBLE,                                // тип
        (rank == 0 ? b.data() : nullptr),          // куда собирать
        (rank == 0 ? recvcountsB.data() : nullptr),// количества
        (rank == 0 ? displsB.data() : nullptr),    // смещения
        MPI_DOUBLE,                                // тип
        0,                                         // root
        MPI_COMM_WORLD                             // коммуникатор
    );

    vector<double> x;                              // вектор решения x

    if (rank == 0) {                               // обратный ход делаю только на root
        x.assign(N, 0.0);                          // выделяю память под решение

        for (int i = N - 1; i >= 0; i--) {         // иду снизу вверх
            double diag = A[i * N + i];            // диагональный элемент

            double sum = 0.0;                      // сумма A[i][j]*x[j]
            for (int j = i + 1; j < N; j++) {      // пробегаю по уже известным x
                sum += A[i * N + j] * x[j];        // накапливаю
            }

            x[i] = (b[i] - sum) / diag;            // вычисляю x[i]
        }
    }

    MPI_Barrier(MPI_COMM_WORLD);                   // синхронизация перед концом времени
    end_time = MPI_Wtime();                        // конец времени

    if (rank == 0) {                               // выводит только root
        cout << "N = " << N << ", processes = " << size << endl; // печатаю параметры

        cout << "Solution x:" << endl;             // заголовок
        for (int i = 0; i < N; i++) {              // печатаю все 8 элементов
            cout << "x[" << i << "] = " << x[i] << endl; // печать элемента
        }

        cout << "Execution time: "                 // печать времени
             << (end_time - start_time)
             << " seconds." << endl;
    }

    MPI_Finalize();                                // завершаю MPI
    return 0;                                      // выхожу из программы
}

Writing program2.cpp


In [6]:
!mpic++ program2.cpp -O2 -o program2
!mpirun --allow-run-as-root --oversubscribe -np 2 ./program2
!mpirun --allow-run-as-root --oversubscribe -np 4 ./program2
!mpirun --allow-run-as-root --oversubscribe -np 8 ./program2

N = 8, processes = 2
Solution x:
x[0] = 0.00609089
x[1] = -0.012175
x[2] = 0.0316275
x[3] = 0.0236677
x[4] = 0.00741247
x[5] = 0.0755191
x[6] = 0.0356818
x[7] = 0.023488
Execution time: 0.000177596 seconds.
N = 8, processes = 4
Solution x:
x[0] = 0.00609089
x[1] = -0.012175
x[2] = 0.0316275
x[3] = 0.0236677
x[4] = 0.00741247
x[5] = 0.0755191
x[6] = 0.0356818
x[7] = 0.023488
Execution time: 0.000588227 seconds.
N = 8, processes = 8
Solution x:
x[0] = 0.00609089
x[1] = -0.012175
x[2] = 0.0316275
x[3] = 0.0236677
x[4] = 0.00741247
x[5] = 0.0755191
x[6] = 0.0356818
x[7] = 0.023488
Execution time: 0.000890414 seconds.


## **Вывод**

В ходе задания была реализована распределенная MPI программа для решения системы линейных уравнений Ax=b методом Гаусса. Процесс с rank = 0 формировал матрицу коэффициентов A размера 8×8 и вектор правых частей b, после чего строки матрицы и соответствующие элементы b распределялись между процессами. Для распределения основной части строк использовалась функция MPI_Scatter, а оставшиеся строки (если N не делится на число процессов) корректно передавались дополнительными сообщениями, поэтому программа сохраняет работоспособность при любом значении -np.

Прямой ход метода Гаусса выполнялся параллельно: на каждом шаге k процесс, владеющий текущей ведущей строкой, подготавливал pivot строку и рассылал ее всем остальным с помощью MPI_Bcast. После этого каждый процесс занулял элементы ниже диагонали только в своих локальных строках. Далее результаты прямого хода (верхнетреугольная матрица и обновленный вектор b) собирались на процессе rank = 0, где выполнялся обратный ход (обратная подстановка) и формировался итоговый вектор решения x.

Корректность решения подтверждается тем, что при разном количестве процессов получен одинаковый вектор x. Это означает, что распределение строк, передача pivot строки через MPI_Bcast и последующая сборка результатов реализованы корректно и дают согласованный результат независимо от количества процессов.

По времени выполнения для N=8 получено:

* -np 2:
t≈0.000178 сек

* -np 4:
t≈0.000588 сек

* -np 8:
t≈0.000890 сек

Увеличение числа процессов в данном эксперименте не приводит к ускорению, так как размер системы очень мал, а основное время начинает уходить на коммуникации и синхронизацию между процессами (рассылка pivot строки и сбор данных), а не на вычисления. Такой результат ожидаем для маленьких N. Реальный выигрыш от MPI обычно проявляется при существенно больших размерах матриц, когда вычислительная нагрузка доминирует над накладными расходами обмена данными.

## Задание 3

In [7]:
%%writefile program3.cpp
#include <mpi.h>                     // подключаю MPI, чтобы работало mpirun и обмен между процессами
#include <iostream>                  // для вывода cout
#include <vector>                    // для vector
#include <random>                    // для генерации случайного графа
#include <limits>                    // для "бесконечности" INF
#include <algorithm>                 // для min

using namespace std;                 // чтобы не писать std:: каждый раз

int main(int argc, char** argv) {                         // стандартный main для MPI

    MPI_Init(&argc, &argv);                               // инициализирую MPI

    int rank;                                             // номер процесса
    int size;                                             // количество процессов
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);                 // узнаю rank
    MPI_Comm_size(MPI_COMM_WORLD, &size);                 // узнаю size

    int N = 8;                                            // размер графа по умолчанию
    if (argc >= 2) N = atoi(argv[1]);                     // если в командной строке дали N, то беру его
    if (N <= 0) {                                         // защита от неправильного N
        if (rank == 0) cout << "N must be positive\n";     // печатаю ошибку только на root
        MPI_Finalize();                                   // завершаю MPI
        return 1;                                         // выхожу с кодом ошибки
    }

    const int INF = 1000000000;                           // большое число вместо бесконечности

    int base = N / size;                                  // сколько строк гарантированно каждому процессу
    int rem  = N % size;                                  // остаток строк

    int local_rows = base + (rank < rem ? 1 : 0);          // сколько строк именно у этого процесса

    int start_row;                                        // глобальный индекс первой строки у процесса
    if (rank < rem) {                                     // если процесс среди первых rem
        start_row = rank * (base + 1);                    // у них блоки на 1 строку больше
    } else {                                              // иначе
        start_row = rem * (base + 1) + (rank - rem) * base;// старт после "длинных" блоков
    }

    vector<int> G;                                        // полная матрица графа будет только у rank 0
    if (rank == 0) {                                      // только root создаёт граф
        G.resize(N * N);                                  // выделяю N*N элементов

        mt19937 rng(42);                                  // фиксированный seed, чтобы результаты повторялись
        uniform_int_distribution<int> wdist(1, 20);        // веса рёбер от 1 до 20
        uniform_int_distribution<int> edist(0, 99);        // шанс наличия ребра

        for (int i = 0; i < N; i++) {                      // иду по строкам
            for (int j = 0; j < N; j++) {                  // иду по столбцам
                if (i == j) {                              // расстояние до себя
                    G[i * N + j] = 0;                      // 0 на диагонали
                } else {                                   // не диагональ
                    int r = edist(rng);                    // генерирую шанс ребра
                    if (r < 60) {                          // 60% что ребро есть
                        G[i * N + j] = wdist(rng);         // задаю вес ребра
                    } else {                               // иначе ребра нет
                        G[i * N + j] = INF;                // ставлю INF
                    }
                }
            }
        }
    }

    vector<int> localG(local_rows * N, INF);               // локальная часть матрицы, только свои строки
    vector<int> fullG;                                     // общий буфер полной матрицы, чтобы делать Allgather
    fullG.resize(N * N, INF);                              // выделяю место под полную матрицу у каждого процесса

    MPI_Barrier(MPI_COMM_WORLD);                           // синхронизация перед замером времени
    double start_time = MPI_Wtime();                       // старт времени

    if (base > 0) {                                        // Scatter делаю только если base > 0
        vector<int> scatterBuf(base * N);                  // временный буфер под base строк

        MPI_Scatter(                                       // раздаю base строк каждому процессу
            (rank == 0 ? G.data() : nullptr),              // root отправляет, остальные nullptr
            base * N,                                      // сколько int отправить каждому
            MPI_INT,                                       // тип int
            scatterBuf.data(),                             // куда принимаю
            base * N,                                      // сколько принимаю
            MPI_INT,                                       // тип
            0,                                             // root
            MPI_COMM_WORLD                                 // коммуникатор
        );

        for (int i = 0; i < base; i++) {                   // копирую base строк в localG
            for (int j = 0; j < N; j++) {                  // копирую каждый элемент строки
                localG[i * N + j] = scatterBuf[i * N + j]; // перенос
            }
        }
    }

    if (rank < rem) {                                      // если процесс получает ещё одну строку
        int local_extra = base;                            // extra строка будет на позиции base

        if (rank == 0) {                                   // root свою extra строку копирует сам
            int global_row = base * size + rank;           // глобальный индекс extra строки
            for (int j = 0; j < N; j++) {                  // копирую строку
                localG[local_extra * N + j] = G[global_row * N + j];
            }
        } else {                                           // остальные получают extra строку через Recv
            MPI_Recv(                                      // принимаю 1 строку длины N
                &localG[local_extra * N],                  // куда кладу
                N,                                         // N элементов
                MPI_INT,                                   // тип int
                0,                                         // от root
                300,                                       // tag
                MPI_COMM_WORLD,                            // коммуникатор
                MPI_STATUS_IGNORE                           // статус не нужен
            );
        }
    }

    if (rank == 0) {                                       // root рассылает extra строки процессам 1..rem-1
        for (int r = 1; r < rem; r++) {                    // только тем, кому нужна extra строка
            int global_row = base * size + r;              // индекс строки
            MPI_Send(                                      // отправляю строку
                &G[global_row * N],                        // адрес строки
                N,                                         // длина N
                MPI_INT,                                   // тип
                r,                                         // кому
                300,                                       // tag
                MPI_COMM_WORLD                              // коммуникатор
            );
        }
    }

    vector<int> recvcounts(size);                          // сколько элементов от каждого для Allgather
    vector<int> displs(size);                              // смещения для Allgather

    int offset = 0;                                        // считаю смещение в fullG
    for (int r = 0; r < size; r++) {                       // пробегаю все процессы
        int r_rows = base + (r < rem ? 1 : 0);             // сколько строк у процесса r
        recvcounts[r] = r_rows * N;                        // сколько элементов он отдаёт
        displs[r] = offset;                                // куда складывать в fullG
        offset += recvcounts[r];                           // увеличиваю offset
    }

    MPI_Allgatherv(                                        // собираю всю матрицу у всех процессов
        localG.data(),                                     // что я отправляю (мои строки)
        local_rows * N,                                    // сколько элементов я отправляю
        MPI_INT,                                           // тип
        fullG.data(),                                      // куда собираю
        recvcounts.data(),                                 // сколько принимать от каждого
        displs.data(),                                     // смещения
        MPI_INT,                                           // тип
        MPI_COMM_WORLD                                     // коммуникатор
    );

    for (int k = 0; k < N; k++) {                           // главный цикл Флойда-Уоршелла по промежуточной вершине k

        for (int li = 0; li < local_rows; li++) {           // иду по локальным строкам, которые принадлежат процессу
            int i = start_row + li;                         // глобальный индекс строки i

            int dik = fullG[i * N + k];                     // расстояние i -> k из полной матрицы

            if (dik >= INF) continue;                       // если i->k бесконечность, нет смысла обновлять

            for (int j = 0; j < N; j++) {                   // иду по столбцам j
                int dkj = fullG[k * N + j];                 // расстояние k -> j
                if (dkj >= INF) continue;                   // если k->j бесконечность, пропускаю
                int candidate = dik + dkj;                  // кандидат на более короткий путь i -> k -> j
                int &dij = localG[li * N + j];              // ссылка на текущий локальный dij
                if (candidate < dij) dij = candidate;       // если кандидат лучше, обновляю
            }
        }

        MPI_Allgatherv(                                     // после обновления локальных строк собираю обновлённую матрицу у всех
            localG.data(),                                  // отправляю свои локальные строки
            local_rows * N,                                 // количество
            MPI_INT,                                        // тип
            fullG.data(),                                   // принимаю полную матрицу
            recvcounts.data(),                              // сколько от каждого
            displs.data(),                                  // смещения
            MPI_INT,                                        // тип
            MPI_COMM_WORLD                                  // коммуникатор
        );
    }

    MPI_Barrier(MPI_COMM_WORLD);                            // синхронизация перед концом времени
    double end_time = MPI_Wtime();                           // конец времени

    vector<int> finalG;                                      // финальная матрица только на root
    if (rank == 0) finalG.resize(N * N);                     // выделяю память на root

    MPI_Gatherv(                                             // собираю финальные строки на root
        localG.data(),                                       // что отправляю
        local_rows * N,                                      // сколько отправляю
        MPI_INT,                                             // тип
        (rank == 0 ? finalG.data() : nullptr),               // куда собирать
        recvcounts.data(),                                   // сколько от каждого
        displs.data(),                                       // смещения
        MPI_INT,                                             // тип
        0,                                                   // root
        MPI_COMM_WORLD                                       // коммуникатор
    );

    if (rank == 0) {                                         // печатаю результат только на root
        cout << "N = " << N << ", processes = " << size << endl;

        cout << "All-pairs shortest paths matrix:" << endl;  // заголовок

        for (int i = 0; i < N; i++) {                        // печатаю матрицу
            for (int j = 0; j < N; j++) {                    // печатаю элемент
                int val = finalG[i * N + j];                 // беру значение
                if (val >= INF / 2) cout << "INF ";          // если далеко, печатаю INF
                else cout << val << " ";                     // иначе печатаю число
            }
            cout << endl;                                    // новая строка
        }

        cout << "Execution time: "                           // печатаю время
             << (end_time - start_time)
             << " seconds." << endl;
    }

    MPI_Finalize();                                          // завершаю MPI
    return 0;                                                // выхожу
}

Writing program3.cpp


In [8]:
!mpic++ program3.cpp -O2 -o program3
!mpirun --allow-run-as-root --oversubscribe -np 2 ./program3 8
!mpirun --allow-run-as-root --oversubscribe -np 4 ./program3 8
!mpirun --allow-run-as-root --oversubscribe -np 8 ./program3 8

N = 8, processes = 2
All-pairs shortest paths matrix:
0 9 6 11 7 12 9 2 
5 0 9 4 8 7 2 7 
6 3 0 7 1 9 5 8 
1 1 7 0 4 3 3 3 
5 2 11 6 0 8 4 7 
6 3 12 7 1 0 5 8 
3 3 7 2 6 5 0 5 
10 7 4 11 5 13 9 0 
Execution time: 0.00210776 seconds.
N = 8, processes = 4
All-pairs shortest paths matrix:
0 9 6 11 7 12 9 2 
5 0 9 4 8 7 2 7 
6 3 0 7 1 9 5 8 
1 1 7 0 4 3 3 3 
5 2 11 6 0 8 4 7 
6 3 12 7 1 0 5 8 
3 3 7 2 6 5 0 5 
10 7 4 11 5 13 9 0 
Execution time: 0.000520693 seconds.
N = 8, processes = 8
All-pairs shortest paths matrix:
0 9 6 11 7 12 9 2 
5 0 9 4 8 7 2 7 
6 3 0 7 1 9 5 8 
1 1 7 0 4 3 3 3 
5 2 11 6 0 8 4 7 
6 3 12 7 1 0 5 8 
3 3 7 2 6 5 0 5 
10 7 4 11 5 13 9 0 
Execution time: 0.00196326 seconds.


## **Вывод**

В ходе задания была реализована MPI программа для параллельного поиска кратчайших путей между всеми парами вершин графа с использованием алгоритма Флойда Уоршелла. Процесс с rank = 0 сформировал матрицу смежности графа G размера 8×8, где веса ребер задавались случайно, а отсутствие ребра обозначалось большим числом INF. Далее строки матрицы были распределены между процессами: основная часть раздавалась через MPI_Scatter, а оставшиеся строки при необходимости передавались дополнительно, поэтому программа корректно работает при любом количестве процессов.

В ходе выполнения алгоритма каждый процесс обновлял только свою часть матрицы расстояний, то есть свои строки. После каждой итерации по промежуточной вершине k процессы обменивались обновленными данными с помощью MPI_Allgather, чтобы у каждого процесса была актуальная полная матрица расстояний и можно было корректно выполнять следующий шаг алгоритма. После завершения всех итераций итоговая матрица была собрана на процессе rank = 0 и выведена на экран.

Корректность реализации подтверждается тем, что при запуске на разном числе процессов получена одинаковая итоговая матрица кратчайших расстояний. Это означает, что распределение данных, локальные обновления и коллективный обмен через MPI_Allgather реализованы правильно и дают согласованный результат независимо от числа процессов.

По измеренному времени выполнения для N=8 получено:

* -np 2:
t≈0.00211 сек

* -np 4:
t≈0.00052 сек

* -np 8:
t≈0.00196 сек

Из результатов видно, что при np=4 время оказалось минимальным, а при np=8 снова увеличилось. Это ожидаемо для маленького графа: вычислительная часть (обновление матрицы) очень небольшая, поэтому на общую длительность сильно влияют накладные расходы MPI, особенно частые обмены MPI_Allgather на каждой итерации k. При слишком большом числе процессов по сравнению с размером задачи рост расходов на коммуникации и синхронизацию может перекрывать выгоду от распараллеливания.

Таким образом, предложенный параллельный подход корректно реализует алгоритм Флойда Уоршелла и сохраняет правильность результата при любом количестве процессов. Наибольший выигрыш по времени ожидается при больших значениях N, когда вычислений становится существенно больше, а относительное влияние коммуникационных затрат уменьшается.